In [11]:
ACTION_SPACE = ('U','D','L','R')

class Grid :  # enviroment

  def __init__(self,rows,cols,start):
    self.rows = rows
    self.cols = cols
    self.i = start[0]
    self.j = start[1]

  def set(self,rewards,actions):
    self.rewards = rewards
    self.actions = actions

  def set_state(self,s):
    self.i = s[0]
    self.j = s[1]

  def current_state(self):
    return (self.i,self.j)

  def is_terminal(self,s):
    return s not in self.actions

  def get_next_state(self,s,a):
    # this answers: where would i end up if i perform action 'a' in state 's'?
    (i,j) = (s[0],s[1])
    # if this action moves you somewhere else, then it will be in this dictionary
    if a in self.actions[(i,j)]:
      if a == 'U':
        i -= 1
      elif a == 'D':
        i += 1
      elif a == 'R':
        i += 1
      elif a == 'L':
        i -= 1
    return (i,j)

  def all_states(self):
    return set(self.actions.keys()) | set(self.rewards.keys())

In [12]:
def standard_grid():
  # .  .  .  1
  # .  x  . -1
  # s  .  .  .
  grid = Grid(3,4,(2,0))
  rewards = {(0,3): 1,(1,3): -1}
  actions = {
      (0,0):('D','R'),
      (0,1):('L','R'),
      (0,2):('L','D','R'),
      (1,0):('U','D'),
      (1,2):('U','D','R'),
      (2,0):('U','R'),
      (2,1):('L','R'),
      (2,2):('L','R','U'),
      (2,3):('L','U'),
      }
  grid.set(rewards, actions)
  return grid

In [13]:
def print_values(V,g):
  for i in range(g.rows):
    print('-' * 40)
    for j in range(g.cols):
      v = V.get((i,j),0)
      if v >= 0 :
        print(" %.2f|" % v, end="")
      else:
        print("%.2f|" % v, end="")
    print("")

def print_policy(P,g):
  for i in range(g.rows):
    print('-' * 40)
    for j in range(g.cols):
      a = P.get((i,j)," ")
      print("  %s  |" % a, end="")
    print("")

In [14]:
grid = standard_grid()

### fixed policy ###
policy = {
    (2,0):'U',
    (1,0):'U',
    (0,0):'R',
    (0,1):'R',
    (0,2):'R',
    (1,2):'U',
    (2,1):'R',
    (2,2):'U',
    (2,3):'L',
    (1,1):'X',
    (1,3):'H',
    (0,3):'G',
  }
print_policy(policy,grid)

----------------------------------------
  R  |  R  |  R  |  G  |
----------------------------------------
  U  |  X  |  U  |  H  |
----------------------------------------
  U  |  R  |  U  |  L  |


In [15]:
V = {}
for s in grid.all_states():
  V[s] = 0

print_values(V,grid)

----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00| 0.00|


In [16]:
transition_probs = {}
rewards = {}

for row in range(grid.rows):
  for col in range(grid.cols):
    state = (row,col)
    if not grid.is_terminal(state):
      for action in ACTION_SPACE:
        next_state = grid.get_next_state(state,action)
        transition_probs[(state,action,next_state)] = 1
        if next_state in grid.rewards:
          rewards[(state,action,next_state)] = grid.rewards[next_state]

print(rewards)
print("-" * 40)
print(transition_probs)

{((2, 3), 'U', (1, 3)): -1, ((2, 3), 'L', (1, 3)): -1}
----------------------------------------
{((0, 0), 'U', (0, 0)): 1, ((0, 0), 'D', (1, 0)): 1, ((0, 0), 'L', (0, 0)): 1, ((0, 0), 'R', (1, 0)): 1, ((0, 1), 'U', (0, 1)): 1, ((0, 1), 'D', (0, 1)): 1, ((0, 1), 'L', (-1, 1)): 1, ((0, 1), 'R', (1, 1)): 1, ((0, 2), 'U', (0, 2)): 1, ((0, 2), 'D', (1, 2)): 1, ((0, 2), 'L', (-1, 2)): 1, ((0, 2), 'R', (1, 2)): 1, ((1, 0), 'U', (0, 0)): 1, ((1, 0), 'D', (2, 0)): 1, ((1, 0), 'L', (1, 0)): 1, ((1, 0), 'R', (1, 0)): 1, ((1, 2), 'U', (0, 2)): 1, ((1, 2), 'D', (2, 2)): 1, ((1, 2), 'L', (1, 2)): 1, ((1, 2), 'R', (2, 2)): 1, ((2, 0), 'U', (1, 0)): 1, ((2, 0), 'D', (2, 0)): 1, ((2, 0), 'L', (2, 0)): 1, ((2, 0), 'R', (3, 0)): 1, ((2, 1), 'U', (2, 1)): 1, ((2, 1), 'D', (2, 1)): 1, ((2, 1), 'L', (1, 1)): 1, ((2, 1), 'R', (3, 1)): 1, ((2, 2), 'U', (1, 2)): 1, ((2, 2), 'D', (2, 2)): 1, ((2, 2), 'L', (1, 2)): 1, ((2, 2), 'R', (3, 2)): 1, ((2, 3), 'U', (1, 3)): 1, ((2, 3), 'D', (2, 3)): 1, ((2, 3), 'L', (1,

In [17]:
iteration = 0
while True:
  for state in grid.all_states():
    if not grid.is_terminal(state):
      new_v = 0
      for action in ACTION_SPACE:
        for next_state in grid.all_states():
          #deteministic actions:
          action_prob = 1 if policy.get(state) == action else 0

          r = rewards.get((state,action,next_state),0)
          new_v += action_prob * transition_probs.get((state,action,next_state),0) * (r + 0.9 * V[next_state])

      V[state] = new_v

  iteration = iteration+1
  print_values(V,grid)
  print("\n")
  if iteration == 10:
    break

----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00|-1.00|


----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00|-1.00|


----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00|-1.00|


----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00|-1.00|


----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00| 0.00|
----------------------------------------
 0.00| 0.00| 0.00|-1.00|


